In [2]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd
from helper import *

main = pd.read_pickle("../clean_main.df")

In [2]:
# Notebook that has some basic feature creation and additions

In [3]:
# Number of horses
df = pd.DataFrame({"race_id":np.unique(main["race_id"])})

raceNum = [len(main.loc[main["race_id"]==r]) for r in df["race_id"] ]
df["num_of_horses"] = raceNum

main = pd.merge(main, df, on="race_id").drop(['index'], axis=1)
main.to_pickle("../clean_main.df")

KeyboardInterrupt: 

In [ ]:
# Horse win percentage
horses = np.unique(main["horse_id"])
winPcts = np.zeros(len(main))
c = 0
for horse in horses:
    printProgress(c, len(horses))
    c += 1
    runs = main.loc[main["horse_id"]==horse]
    runsI = runs.index
    
    for i in range(len(runs)):
        if i == 0: # First run
            winPct = 0.
        else:
            prevRuns = runs.iloc[:i, :]
            wins = sum(prevRuns["won"])
            winPct = float(wins) / prevRuns.shape[0]
            
        runIndex = runsI[i]
        winPcts[runIndex] = winPct
        
main["horse_win_percent"] = winPcts
main.to_pickle("../clean_main.df")

In [ ]:
# Trainer win percentage
trainers = np.unique(main["trainer_id"])
winPcts = np.zeros(len(main))
c = 0.
for trainer in trainers:
    printProgress(c, len(trainers), jump=10)
    c += 1
    runs = main.loc[main["trainer_id"]==trainer]
    runsI = runs.index
    
    for i in range(len(runs)):
        if i == 0: # First run
            winPct = 0.
        else:
            prevRuns = runs.iloc[:i, :]
            wins = sum(prevRuns["won"])
            winPct = float(wins) / prevRuns.shape[0]
            
        runIndex = runsI[i]
        winPcts[runIndex] = winPct
        
main["trainer_win_percent"] = winPcts
main.to_pickle("../clean_main.df")

In [ ]:
# Jockey win percentage
jockeys = np.unique(main["jockey_id"])
winPcts = np.zeros(len(main))
c = 0.
for jockey in jockeys:
    printProgress(c, len(jockeys), jump=10)
    c += 1
    runs = main.loc[main["jockey_id"]==jockey]
    runsI = runs.index
    
    for i in range(len(runs)):
        if i == 0: # First run
            winPct = 0.
        else:
            prevRuns = runs.iloc[:i, :]
            wins = sum(prevRuns["won"])
            winPct = float(wins) / prevRuns.shape[0]
            
        runIndex = runsI[i]
        winPcts[runIndex] = winPct
        
main["jockey_win_percent"] = winPcts
main.to_pickle("../clean_main.df")

In [ ]:
# Add days since last race for a horse
horses = np.unique(main["horse_id"])
days = np.zeros(len(main))
c = 0
vals = 0.
count = 0
for horse in horses:
    printProgress(c, len(horses))
    c += 1
    runs = main.loc[main["horse_id"]==horse]
    runsI = runs.index
    
    for i in range(len(runs)):
        if i == 0: # First run
            daysSince = 0.
        else:
            prevRun = runs.iloc[i-1, :]
            currentRun = runs.iloc[i,:]
            prevDate = prevRun.date
            currentDate = currentRun.date
            
            daysSince = int((currentDate - prevDate).days)
            vals += daysSince
            count +=1 
            
        runIndex = runsI[i]
        days[runIndex] = daysSince
        
avg = vals / count
for i in range(len(days)):
    if days[i] == 0:
        days[i] = avg
    
        
main["days_since_last_run"] = days
main.to_pickle("../clean_main.df")

In [ ]:
# Add average speed rating for this horse
horses = np.unique(main["horse_id"])
avgs = np.zeros(len(main))
last = np.zeros(len(main))
c = 0
for horse in horses:
    printProgress(c, len(horses), jump=100)
    c += 1
    runs = main.loc[main["horse_id"]==horse]
    runsI = runs.index
    
    for i in range(len(runs)):
        if i == 0: # First run
            avg = 80.
            lastRating = 80.
        else:
            prevRuns = runs.iloc[:i, :]
            avg = prevRuns['speed_rating'].mean()
            lastRating = prevRuns.iloc[i-1,:]["speed_rating"]
            
        runIndex = runsI[i]
        avgs[runIndex] = avg
        last[runIndex] = lastRating
        
main["average_speed_rating"] = avgs
main["last_speed_rating"] = last
main.to_pickle("../clean_main.df")

In [ ]:
# Add best speed figure on this surface and distance
horses = np.unique(main["horse_id"])
bestSD = np.zeros(len(main))
c = 0
count = 0
for horse in horses:
    printProgress(c, len(horses), jump=150)
    c += 1
    runs = main.loc[main["horse_id"]==horse]
    runsI = runs.index
    
    for i in range(len(runs)):
        if i == 0: # First run
            best = 80.
            count += 1
        else:
            prevRuns = runs.iloc[:i, :]
            currentRun = runs.iloc[i,:]
            dist, surf = currentRun.distance, currentRun.surface
            prevSDRuns = prevRuns.loc[(prevRuns.distance==dist) & (prevRuns.surface == surf)]
            if prevSDRuns.shape[0] != 0:
                best = max(prevSDRuns["speed_rating"])
            else:
                best = currentRun['average_speed_rating']
                count += 1
            
        runIndex = runsI[i]
        bestSD[runIndex] = best
        
print(count / float(len(main)))
main["best_surface_distance"] = bestSD 
main.to_pickle("../clean_main.df")

In [ ]:
# Add change of equipment
horses = np.unique(main["horse_id"])
changeOfE= np.zeros(len(main))
c = 0
for horse in horses:
    printProgress(c, len(horses), jump=150)
    c += 1
    runs = main.loc[main["horse_id"]==horse]
    runsI = runs.index
    
    for i in range(len(runs)):
        if i == 0: # First run
            change = 1 # Say that it was a change for the first timers
        else:
            prevRun = runs.iloc[i-1, :]
            currentRun = runs.iloc[i,:]
            if prevRun.horse_gear != currentRun.horse_gear:
                change = 1
            else:
                change = 0
            
            
        runIndex = runsI[i]
        changeOfE[runIndex] = change
        
main["gear_change"] = changeOfE
main.to_pickle("../clean_main.df")

In [ ]:
# Add a normalized results column
main["norm_result"] = main.apply(lambda x: x.result / float(x.num_of_horses), axis=1)
main.to_pickle("../clean_main.df")

In [ ]:
# Add performance on going
FAST = ["GOOD TO FIRM", "GOOD", "FAST"]
MEDIUM = ["YIELDING", "GOOD TO YIELDING", "YIELDING TO SOFT"]
SLOW = ["SOFT", "WET", "SLOW"]

horses = np.unique(main["horse_id"])
goingRecord= np.zeros(len(main))
c = 0
for horse in horses:
    printProgress(c, len(horses), jump=150)
    c += 1
    runs = main.loc[main["horse_id"]==horse]
    runsI = runs.index
    
    for i in range(len(runs)):
        if i == 0: # First run
            performance = 80 
        else:
            prevRuns = runs.iloc[:i, :]
            currentRun = runs.iloc[i,:]
            
            going = currentRun["going"]
            if going in FAST:
                thisGoing = FAST
            elif going in MEDIUM:
                thisGoing = MEDIUM
            else:
                thisGoing = SLOW
                
            prevGoingRuns = prevRuns.loc[prevRuns.going.isin(thisGoing)]
            
            if prevGoingRuns.shape[0] == 0:
                performance = 80
            else:
                performance = prevGoingRuns["speed_rating"].mean()
            
            
        runIndex = runsI[i]
        goingRecord[runIndex] = performance
        
main["going_rating"] = goingRecord
main.to_pickle("../clean_main.df")

In [ ]:
# Add normalized results for trainer, jockey, horse
def addNormRecord(var):
    varName = var + "_id"
    temps = np.unique(main[varName])
    
    records = np.zeros(len(main))
    c = 0
    for temp in temps:
        printProgress(c, len(temps), jump=150)
        c += 1
        races = main.loc[main[varName]==temp]
        raceI = races.index
        
        for i in range(len(races)):
            if i == 0:
                record = 0.5
            else:
                pastRaces = races.iloc[:i, :]
                record = pastRaces["norm_result"].mean()
                
            records[raceI[i]] = record
        
    main[var+"_norm_record"] = records
    main.to_pickle("../clean_main.df")
        
addNormRecord("horse")
addNormRecord("trainer")
addNormRecord("jockey")

In [7]:
# Add average speed rating (specials) for this horse
horses = np.unique(main["horse_id"])
avgs_1 = np.zeros(len(main))
last_1 = np.zeros(len(main))
avgs_last = np.zeros(len(main))
last_last = np.zeros(len(main))
avgs_ratio = np.zeros(len(main))
last_ratios = np.zeros(len(main))
c = 0
for horse in horses:
    printProgress(c, len(horses), jump=100)
    c += 1
    runs = main.loc[main["horse_id"]==horse]
    runsI = runs.index
    
    for i in range(len(runs)):
        if i == 0: # First run
            avg_1 = 80.
            lastRating_1 = 80.
            avg_last = 80.
            lastRating_last = 80.
            avg_ratio = 80.
            last_ratio = 80.
        else:
            prevRuns = runs.iloc[:i, :]
            avg_1 = prevRuns['sec1_speed_rating'].mean()
            lastRating_1 = prevRuns.iloc[i-1,:]["sec1_speed_rating"]
            
            avg_last = prevRuns['last_sec_speed_rating'].mean()
            lastRating_last = prevRuns.iloc[i-1,:]["last_sec_speed_rating"]
            
            avg_ratio = prevRuns["speed_rating_ratio"].mean()
            last_ratio = prevRuns.iloc[i-1,:]["speed_rating_ratio"]
            
            
        runIndex = runsI[i]
        avgs_1[runIndex] = avg_1
        last_1[runIndex] = lastRating_1
        avgs_last[runIndex] = avg_last
        last_last[runIndex] = lastRating_last
        avgs_ratio[runIndex] = avg_ratio
        last_ratios[runIndex] = last_ratio
        
main["avg_sec1_speed_rating"] = avgs_1
main["last_sec1_speed_rating"] = last_1
main["avg_last_sec_speed_rating"] = avgs_last
main["last_last_sec_speed_rating"] = last_last
main["avg_speed_rating_ratio"] = avgs_ratio
main["last_speed_rating_ratio"] = last_ratios
main.to_pickle("../clean_main.df")

0.00% Complete ...
2.31% Complete ...
4.62% Complete ...
6.93% Complete ...
9.24% Complete ...
11.55% Complete ...
13.86% Complete ...
16.17% Complete ...
18.48% Complete ...
20.79% Complete ...
23.09% Complete ...
25.40% Complete ...
27.71% Complete ...
30.02% Complete ...
32.33% Complete ...
34.64% Complete ...
36.95% Complete ...
39.26% Complete ...
41.57% Complete ...
43.88% Complete ...
46.19% Complete ...
48.50% Complete ...
50.81% Complete ...
53.12% Complete ...
55.43% Complete ...
57.74% Complete ...
60.05% Complete ...
62.36% Complete ...
64.67% Complete ...
66.97% Complete ...
69.28% Complete ...
71.59% Complete ...
73.90% Complete ...
76.21% Complete ...
78.52% Complete ...
80.83% Complete ...
83.14% Complete ...
85.45% Complete ...
87.76% Complete ...
90.07% Complete ...
92.38% Complete ...
94.69% Complete ...
97.00% Complete ...
99.31% Complete ...


In [4]:
horses = np.unique(main["horse_id"])
numOfRaces = np.zeros(len(main))
c = 0
for horse in horses:
    printProgress(c, len(horses))
    c += 1
    runs = main.loc[main["horse_id"]==horse]
    runsI = runs.index
    
    for i in range(len(runs)):   
        runIndex = runsI[i]
        numOfRaces[runIndex] = i
        
main["num_of_races"] = numOfRaces
main.to_pickle("../clean_main.df")

0.00% Complete ...
23.09% Complete ...
46.19% Complete ...
69.28% Complete ...
92.38% Complete ...
